In [56]:
import os
from transformers import AutoModel, AutoTokenizer
import requests
from google.cloud import translate_v3

In [57]:
os.environ['HF_TOKEN'] = os.environ.get("HF_TOKEN")
PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT")
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS")

In [58]:
def translate_text(text: str, source_language_code: str, target_language_code: str) -> str:
    client = translate_v3.TranslationServiceClient()
    parent = f"projects/{PROJECT_ID}/locations/global"

    response = client.translate_text(
        contents=[text],
        target_language_code=target_language_code,
        parent=parent,
        mime_type="text/plain",
        source_language_code=source_language_code,
    )
    
    translated_text = response.translations[0].translated_text

    return translated_text

In [59]:
def query_model(text: str):
    headers = {
        'Authorization': f'Bearer {os.getenv("HF_TOKEN")}'
    }
    API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
    payload = {
        "inputs": text,
        "parameters": {"return_full_text": False},
        "options": {"use_cache": True}
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

In [60]:
def main():
    text_to_translate = input("Enter your question in Sinhala: ")
    
    translated_input = translate_text(text=text_to_translate, source_language_code="si", target_language_code="en-US")
    result = query_model(translated_input)
    
    if isinstance(result, list) and result:
        model_output = result[0].get('generated_text', 'No text generated')
        model_output = model_output.replace('\n', ' ').strip()
    else:
        model_output = "No valid response received from the model"
    
    translated_output = translate_text(text=model_output, source_language_code="en-US", target_language_code="si")
    print("Final Translated Output:", translated_output)

In [ ]:
if __name__ == "__main__":
    main()